In [ ]:
import great_expectations as gx
from great_expectations.checkpoint import SimpleCheckpoint
from great_expectations.core.batch import BatchRequest
from great_expectations.core.yaml_handler import YAMLHandler
from great_expectations.datasource.fluent.interfaces import DataAsset
from great_expectations.core.expectation_configuration import ExpectationConfiguration
from great_expectations.datasource.fluent import SqliteDatasource

In [ ]:
yaml = YAMLHandler()

In [ ]:
context = gx.get_context(context_root_dir="./great_expectations/", cloud_mode=False)

In [ ]:
sqlite_database_path = "../../../data/visits.db"

In [ ]:
datasource: SqliteDatasource = context.sources.add_or_update_sqlite(
    name="visits_datasource", connection_string=f"sqlite:///{sqlite_database_path}"
)

In [ ]:
# validator
table_asset = datasource.add_table_asset(name="visits_asset", table_name="event_names")

In [ ]:
batch_request: BatchRequest = table_asset.build_batch_request()

In [ ]:
expectation_suite_name = "visitors_exp"

In [ ]:
expectation_suite = context.add_or_update_expectation_suite(
    expectation_suite_name=expectation_suite_name
)

In [ ]:
new_config = ExpectationConfiguration(
    expectation_type="expect_column_values_to_be_in_set",
    kwargs={"column": "event_type", "value_set": ["page_load", "page_view"]},
)

In [ ]:
expectation_suite.add_expectation(new_config)

In [ ]:
expectation_suite = context.add_or_update_expectation_suite(
    expectation_suite=expectation_suite
)

In [ ]:
expectation_suite

In [ ]:
checkpoint_config: dict = {
    "class_name": "Checkpoint",
    "name": "my_checkpoint",
    "config_version": 1,
    "module_name": "great_expectations.checkpoint",
    "run_name_template": "%Y-%M-foo-bar-template",
    "action_list": [
        {
            "name": "store_validation_result",
            "action": {
                "class_name": "StoreValidationResultAction",
            },
        },
        {
            "name": "store_evaluation_params",
            "action": {
                "class_name": "StoreEvaluationParametersAction",
            },
        },
        {
            "name": "update_data_docs",
            "action": {
                "class_name": "UpdateDataDocsAction",
            },
        },
    ],
    "validations": [
        {
            "batch_request": batch_request,
            "expectation_suite_name": "visitors_exp",
        }
    ],
    "runtime_configuration": {},
}
context.add_or_update_checkpoint(**checkpoint_config)

In [ ]:
result_format: dict = {
    "result_format": "COMPLETE",
    "unexpected_index_column_names": ["event_id"],
}

In [ ]:
results = context.run_checkpoint(
    checkpoint_name="my_checkpoint", result_format=result_format
)

In [ ]:
evrs = results.list_validation_results()

In [ ]:
evrs[0]["results"][0]["result"]

In [ ]:
context.open_data_docs()